**[Часть 2]** 

    а) лемматизировать текст с помощью mystem или pymorphy2,
    б) найти такие леммы (не исходные словоформы), в которых было бы две (не больше, не меньше) буквы «о», 
    в) программно обратиться к странице (скачать питоном) http://lib.ru/POEZIQ/PESSOA/lirika.txt , 
    г) составить словарь для текста, который там размещен,
    д) записать этот словарь в файл в формате JSON.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks')

In [3]:
pip install pymorphy2

# Часть 1

In [ ]:
#pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=863a16b1f729ee64e4756886118797651005ef9424045c9692f849eeb0728565
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
#import wget

#wget.download('https://cloclo15.cloud.mail.ru/weblink/view/V2pn/2yB9UEnz2?etag=1A0C163E53183EAA3076A31BD641FFC40534B7B6', 'cloud_text.txt')

'cloud_text.txt'

Обработка текста и создание словаря

In [4]:
f = open('cloud_text.txt', 'r', encoding = 'utf-8-sig')
text = f.read()
f.close()

In [5]:
text[:100]

'\n\n\n\n\n\n\n\n\nАРВО МЕТС\n\n\n В осенних лесах.\n\nизбранное\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nстихотворения\nСудьба русского по'

In [6]:
import re

rewords = re.compile('[\\n]', re.DOTALL)
text = re.sub(rewords, ' ', text).lower() #заменяем лишние символы на пробелы, чтобы слова не склеились, и приводим все к нижнему регистру
text = re.sub("[^\s\w]+", " ", text) #убираем всю пунктуцацию

In [7]:
text[:100]

'         арво метс    в осенних лесах   избранное                   стихотворения судьба русского по'

In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from nltk import word_tokenize

words = word_tokenize(text, language = 'russian')

In [10]:
words[:5]

['арво', 'метс', 'в', 'осенних', 'лесах']

или можно выделить слова при помощи регулярных выражений

In [11]:
words_2 = [w for w in re.findall("([A-Za-zА-Яа-яЁё]+)", text)]

In [12]:
words_2[:5]

['арво', 'метс', 'в', 'осенних', 'лесах']

Составим словарь

In [13]:
import collections
c = collections.Counter()
for word in words:
  c[word] += 1

In [ ]:
from collections import OrderedDict

f = open('test_text_dict.csv', 'w', encoding='utf-8')
dd = OrderedDict(sorted(dict(c).items(), key=lambda t: t[1], reverse = True))
for i in dd.keys():
  s = i+','+str(dd[i])+'\n'
  f.write(s)
f.close()

# Часть 2.1

Лемматизируем слова из получившегося в первой части списка

In [14]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

lemmas = [morph.parse(word)[0].normal_form for word in words]

In [15]:
for i in range(10):
  print(words[i], lemmas[i])

арво арво
метс метс
в в
осенних осенний
лесах леса
избранное избранный
стихотворения стихотворение
судьба судьба
русского русский
поэта поэт


In [26]:
f = open("oo_list.txt", 'w', encoding = 'utf-8')
for lemma in lemmas:
  if lemma.count('о') == 2:
    s = lemma + '\n'
    f.write(s)
f.close()

# Часть 2.2

In [16]:
import requests

r = requests.get('http://lib.ru/POEZIQ/PESSOA/lirika.txt')  

In [17]:
r.text[:100]

'<html><head><title>Фернандо Пессоа. Лирика</title></head><body><pre><div align=right><form action=/P'

In [18]:
reContents = re.compile('<pre>.*?<pre>', re.DOTALL) #уберем содержание меню в верхнем углу
reTag = re.compile('<.*?>', re.DOTALL) #очистим текст от тегов
text = re.sub(reContents, '', r.text)
text = re.sub(reTag, '', text)

In [19]:
text[:100]

'Фернандо Пессоа. Лирика\nФернандо Пессоа. Лирика\n\n     ББК 84.4ПР, П28\n     Составление Е. ВИТКОВСКОГ'

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.tokenize import RegexpTokenizer #используем другой токенизатор
from nltk.corpus import stopwords #уберем также стоп-слова

morph = pymorphy2.MorphAnalyzer()
token = RegexpTokenizer('\w+')
stops = set(stopwords.words('russian')) 

def text_to_lemmas(text):
    lemmas = [morph.parse(word)[0].normal_form for word in token.tokenize(text) if morph.parse(word)[0].normal_form not in stops]
    return lemmas

In [22]:
import collections
from collections import OrderedDict

def freq_dict(words):
  c = collections.Counter()
  for word in words:
    c[word] += 1
  
  return OrderedDict(sorted(dict(c).items(), key=lambda t: t[1], reverse = True))

In [23]:
final_dict = freq_dict(text_to_lemmas(text))

In [81]:
import json

with open('test_text_dict_2.json', 'w') as f:
    json.dump(final_dict, f)